In [10]:
import sympy as sy
import math
import numpy as np

In [53]:
class SymbolicModel:
    def __init__(self,eqs,pars=None):
        import simpy as sy
        self.eqs = eqs
        self.pars = None
        self._X = None
        self._F = None
        self._loadEqs(eqs)
        return None
        
    def _loadEqs(self,dict):
        """_summary_
        This function should initialize symbolic parameters
        for the functions
        """
        symbolic_vars = []
        symbolic_eqs = []
        for var in self.eqs.keys():
            exec(f"self.{var} = sy.sympify('{var}')")
            exec(f"symbolic_vars.append(self.{var})")
            exec(f"symbolic_eqs.append(sy.sympify(self.eqs['{var}']))")
        self._X = sy.Matrix(symbolic_vars)
        self._F = sy.Matrix(symbolic_eqs)

    def loadPars(self,pars):
        self.pars = pars.copy()
        
    def convertToExpr(self):
        for var,eq in self.eqs.items():
            _expr = sy.sympify(eq)
            self.expr[var]=_expr

    def evaluateParams(self):
        F_s = self._F.subs(self.pars)
        return F_s
    
    def getEqs(self):
        return None

    def _rk2(self):
        pass

    def simulate(self):
        return None
    
    def _lamdify(self,expr):
        return ()
    
    def saveExpressions():
        pass
    
    def _getJacobian(self):
        return self._F.jacobian(self._X).subs(self.pars)
    
    def evaluateJacobian(self,d:dict):
        jacobian = self._getJacobian()
        j_np = np.array(jacobian.subs(d)).astype('float64')
        return j_np.astype('float64')
    
    def calculateEigenValues(self,X):
        eiv,_ = np.linalg.eig(X)
        if eiv.shape[0] == 2:
            l1,l2 = eiv
            tau = l1+l2
            delta = li*l2
            stability = _assignStability(tau,delta)
            return stability
        return None
    
    @staticmethod
    def _assignStability(tau,delta):
        if tau < 0:
            return 'saddle point'
        elif tau == 0:
            return 'non isolated fixed point'
        # up to this point means tau is positive
        if tau**2 - 4*delta < 0:
            if delta > 0:
                return 'unstable node'
            elif delta < 0:
                return 'stable node'
            else: 
                return 'center'
        elif tau**2 - 4*delta > 0:
            if delta > 0:
                return 'unstable spiral'
            elif delta < 0:
                return 'stable spiral'
            else: 
                return 'center'
        else return 'degenerate node'
        

In [54]:
phi_x = lambda x: f"exp(b_{x}*(v - v_{x})) - exp((b_{x} - 1)*(v - v_{x}))"
phi_u = phi_x('u')
phi_d = phi_x('d')
m_infty = "(1 + exp(g_m*(v_m - v)))"
J_d = f"a_d*{m_infty}*(1-w)*({phi_d})" 
J_u = f"a_u*w*{phi_u}"
dtv = f"-{J_u} - {J_d}"
print(dtv)
#
alpha = "r_w*(exp(b_w*g_w*(v-v_w)))" 
beta =  "r_w*(exp((b_w-1)*g_w*(v-v_w)))"
dtw = f"(w**k)*({alpha}*(1-w) - {beta}*w)"
v_infty = f""
display(dtw)

-a_u*w*exp(b_u*(v - v_u)) - exp((b_u - 1)*(v - v_u)) - a_d*(1 + exp(g_m*(v_m - v)))*(1-w)*(exp(b_d*(v - v_d)) - exp((b_d - 1)*(v - v_d)))


'(w**k)*(r_w*(exp(b_w*g_w*(v-v_w)))*(1-w) - r_w*(exp((b_w-1)*g_w*(v-v_w)))*w)'

In [55]:
eqs = {'v':dtv,
        'w':dtw
        }
pars = {'a_u':1.0, 
        'a_d':0.3, 
        'g_m':6, 
        'g_w':-4, 
        'v_w':0, 
        'v_m':-20,# falta normalizar V_T = 26.7 
        'b_w':0.6,
        'r_w':1,
         'k' :0, #[0,1]
        'b_u':0.5,#[0,1]
        'b_d':0.5,#[0,1]
        'v_u':60,#duda
        'v_d':-90,#duda
        'g_m':0.5,#duda
        # 'v':'v_infty'
        }

#primera prueba sin sustituir parámetros.
#objetivo: obtener un jacobiano
sm = SymbolicModel(eqs)
sm.loadPars(pars)
F_vw = sm.evaluateParams()
#%%
F_vw

Matrix([
[-9.35762296884017e-14*w*exp(0.5*v) - 0.3*(1 - w)*(1 + 4.53999297624849e-5*exp(-0.5*v))*(3.49342710574851e+19*exp(0.5*v) - 2.86251858054939e-20*exp(-0.5*v)) - 10686474581524.5*exp(-0.5*v)],
[                                                                                                                                                       -w*exp(1.6*v) + (1 - w)*exp(-2.4*v)]])

In [56]:
J = sm.getJacobian()
#%%
sm._loadEqs(dict)
sm.loadPars(pars)
#sm.convertToExpr()
print(sm.eqs)
print(sm.pars)
display(sm.evaluateParams())
#sm._J



AttributeError: 'SymbolicModel' object has no attribute 'getJacobian'

In [58]:
Jn = sm.evaluateJacobian({'v':-11,'w':0})
display(Jn)

array([[-2.01078101e+16,  4.33063184e+16],
       [-7.00786237e+11, -2.91994265e+11]])

In [60]:
eiv,eim = np.linalg.eig(Jn)

In [63]:
eiv.shape[0]

2

In [64]:
l1,l2 = eiv

In [65]:
l1

-2.010630069697454e+16